In [1]:
import sys
from pathlib import Path
from os import getcwd
sys.path.append(str(Path(getcwd()).parent))


from bulletin import root, default_input, default_output
from bulletin.services.metabase import Metabase
from bulletin.systems.notifica import Notifica
from unidecode import unidecode as unidecode

import pandas as pd


---------------------------------------------------------------

root: c:\SESA\dev\bulletin
parent: c:\SESA\dev
default_input: c:\SESA\dev\input
default_output: c:\SESA\dev\output

---------------------------------------------------------------



In [2]:
# notifica = Notifica(usecols=False)
# fator_risco = notifica.schema.loc[notifica.schema['group_name']=='comorbidades']

In [3]:
mb = Metabase()
mb.generate_notifica_query('instituicao',usecols=False,group='instituicao')

limit: 100000
Cookie:'13646********-****-****-****-************27d89'

sql_files:
	0: all_notifica
	1: coisa
	2: comorbidades
	3: diario
	4: new_query
	5: old_diario
	6: query_linda
	7: usecols_notifica

sql_results:
	0: comorbidades_len
	1: diario_len
	2: query_linda_len
Select 10 columns


'SELECT id, situacao_trabalho, doenca_trabalho, paciente_institucionalizado, instituicao, tipo_paciente_institucionalizado, nome_empresa, uf_empresa, ibge_empresa, instituicao_ensino FROM notificacao WHERE classificacao_final = 2 AND excluir_ficha = 2 AND status_notificacao in (1,2) ORDER BY 1 LIMIT ALL OFFSET 0'

In [4]:
mb.list_sql_files()


sql_files:
	0: all_notifica
	1: coisa
	2: comorbidades
	3: diario
	4: instituicao
	5: new_query
	6: old_diario
	7: query_linda
	8: usecols_notifica


In [7]:
# notifica_instituicao = mb.download_notificacao('instituicao',normalize=False)

Download instituicao

Requesting csv
Success code 200
Saving query in c:\SESA\dev\input\queries\instituicao_len.csv
Download finish!
time elapsed: 0:04:28.348252
Columns size: 1


query_size: 1172190

select ... limit 100000 offset 0

Requesting csv
Success code 200
Saving query in c:\SESA\dev\input\queries\tmp\instituicao_0_100000.csv
Download finish!
time elapsed: 0:05:34.756514
Columns size: 10

select ... limit 100000 offset 100000

Requesting csv
Success code 200
Saving query in c:\SESA\dev\input\queries\tmp\instituicao_100000_200000.csv
Download finish!
time elapsed: 0:05:09.125676
Columns size: 10

select ... limit 100000 offset 200000

Requesting csv
Success code 200
Saving query in c:\SESA\dev\input\queries\tmp\instituicao_200000_300000.csv
Download finish!
time elapsed: 0:04:41.173784
Columns size: 10

select ... limit 100000 offset 300000

Requesting csv
Success code 200
Saving query in c:\SESA\dev\input\queries\tmp\instituicao_300000_400000.csv
Download finish!
time elapsed

In [8]:
# notifica_instituicao.save()

In [11]:
notifica = Notifica('notifica')

In [17]:
notifica.load()
notifica_df = notifica.df

In [18]:
notifica_instituicao.load()
df_temp = notifica_instituicao.df

In [21]:
notifica_df = notifica_df.merge(how = 'left', on = 'id', right = df_temp)

In [40]:
def normalize_hash(string: str):
    return "".join(filter(lambda char: 'A' <= char <= 'Z', str(string).upper()))

def normalize_text(text):
    x = str(text)
    x = unidecode(x)
    x = normalize_hash(x) 
    if x == 'nan' or len(x) == 0 or x == '0' or x == 'nao_informado':
        return ''
    else:
        return x

In [41]:
notifica_df['nome_empresa_hash'] = notifica_df['nome_empresa'].apply(lambda row: normalize_text(row))

In [42]:
sesa_df = notifica_df.loc[(notifica_df['nome_empresa_hash'].notna()) & (notifica_df['nome_empresa_hash'].str.contains('SECRETARIADEESTADODASAUDE') | notifica_df['nome_empresa_hash'].str.contains('SESA') | notifica_df['nome_empresa_hash'].str.contains('REGIONALDESAUDE'))]

In [52]:
sesa_df.loc[sesa_df['evolucao'] == 2]

744492    68
Name: idade, dtype: int64

In [47]:
sesa_df.to_excel('casos_sesa.xlsx', index=False)

In [48]:
sesa_df.shape

(58, 62)